In [1]:
import pandas as pd

file_path = 'API Call Dataset (1).csv'
data = pd.read_csv(file_path)

top_apis = data['API Code'].value_counts().head(4).index.tolist()

for api in top_apis:
    api_data = data[data['API Code'] == api]
    file_name = f"top_{api}_usage.csv"
    api_data.to_csv(file_name, index=False)
    print(f"Saved {file_name}")

Saved top_A9_usage.csv
Saved top_A2_usage.csv
Saved top_A7_usage.csv
Saved top_A4_usage.csv


In [2]:
pip install pandas numpy tensorflow scikit-learn matplotlib

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf

data = pd.read_csv("top_A2_usage.csv")
data['Time of Call'] = pd.to_datetime(data['Time of Call'], dayfirst=True)
data.set_index('Time of Call', inplace=True)

time_series = data.resample('H').count()['API Code']

scaler = MinMaxScaler()
time_series_scaled = scaler.fit_transform(time_series.values.reshape(-1, 1))

def create_supervised(data, lag=24):
    X, y = [], []
    for i in range(lag, len(data)):
        X.append(data[i-lag:i])
        y.append(data[i])
    return np.array(X), np.array(y)

lag = 24
X, y = create_supervised(time_series_scaled, lag)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


<ipython-input-3-ef53207c39ad>:13: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_series = data.resample('H').count()['API Code']


In [4]:
from tensorflow.keras.layers import GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def create_gru_model(input_dim):
    model = Sequential()
    model.add(GRU(50, activation='relu', input_shape=input_dim))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def create_mlp_model(input_dim):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(input_dim[0],)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def create_lstm_model(input_dim):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=input_dim))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model


In [7]:
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def create_fcn_model(input_dim):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=input_dim))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [8]:
from tensorflow.keras.layers import Add, BatchNormalization, Conv1D, GlobalAveragePooling1D, Dense
from tensorflow.keras import Input, Model
import tensorflow as tf

def create_resnet_model(input_dim):
    input_tensor = Input(shape=input_dim)
    y = Conv1D(64, kernel_size=3, padding='same', activation='relu')(input_tensor)
    y = BatchNormalization()(y)
    residual_connection = y

    for _ in range(3):
        y = Conv1D(64, kernel_size=3, padding='same', activation='relu')(y)
        y = BatchNormalization()(y)
        y = Add()([y, residual_connection])
        residual_connection = y

    y = GlobalAveragePooling1D()(y)
    output_tensor = Dense(1)(y)
    model = Model(inputs=input_tensor, outputs=output_tensor)
    model.compile(optimizer='adam', loss='mse')
    return model


In [12]:
from math import sqrt
from sklearn.metrics import mean_squared_error

# Function to train and evaluate models
def train_and_evaluate_model(model_creator, X_train_data, y_train_data, X_test_data, y_test_data, model_label):
    model_instance = model_creator(X_train_data.shape[1:])  # Create the model instance
    training_history = model_instance.fit(X_train_data, y_train_data, epochs=20, batch_size=32, verbose=0, validation_data=(X_test_data, y_test_data))  # Train the model
    predictions = model_instance.predict(X_test_data)  # Get predictions from the model
    rmse_value = sqrt(mean_squared_error(y_test_data, predictions))  # Calculate RMSE
    print(f"{model_label} RMSE: {rmse_value}")  # Print RMSE value for each model
    return model_instance, rmse_value  # Return trained model and RMSE

# Model creation functions (ensure these are defined elsewhere in your code)
model_builders = {
    "LSTM": create_lstm_model,
    "GRU": create_gru_model,
    "MLP": create_mlp_model,
    "FCN": create_fcn_model,
    "ResNet": create_resnet_model,
}

# Dictionary to store evaluation results (RMSE for each model)
evaluation_results = {}

# Train and evaluate each model
for model_name, builder_function in model_builders.items():
    trained_model, rmse_score = train_and_evaluate_model(builder_function, X_train, y_train, X_test, y_test, model_name)
    evaluation_results[model_name] = rmse_score  # Store RMSE for each model

# Print the performance of all models
print("Model Performance:")
for model_label, rmse_score in sorted(evaluation_results.items(), key=lambda x: x[1]):  # Sort models by RMSE value
    print(f"{model_label}: RMSE = {rmse_score}")  # Print RMSE for each model


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
LSTM RMSE: 0.19058193641871415


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
GRU RMSE: 0.19065641431494715


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
MLP RMSE: 0.208667167482576


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
FCN RMSE: 0.19064173135229717
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
ResNet RMSE: 0.19747718212228776
Model Performance:
LSTM: RMSE = 0.19058193641871415
FCN: RMSE = 0.19064173135229717
GRU: RMSE = 0.19065641431494715
ResNet: RMSE = 0.19747718212228776
MLP: RMSE = 0.208667167482576


In [13]:
# Select the best model based on RMSE
best_model_name = min(evaluation_results, key=evaluation_results.get)  # Model with the lowest RMSE is considered the best
print(f"Best Model: {best_model_name}")

# Retrain the best model using the entire training dataset
best_model = model_builders[best_model_name](X_train.shape[1:])  # Create the best model instance
best_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)  # Train the model on the entire dataset

# Start with the last observed input from the test set for prediction
current_input = X_test[-1].reshape(1, -1, 1)  # Reshaping the input for the model

# Define time horizons for the prediction (next hour, next day, next week, next month)
horizons = [1, 24, 24 * 7, 24 * 30]  # 1 hour, 24 hours (1 day), 168 hours (1 week), 720 hours (1 month)

# Initialize an empty list to store the future predictions
future_predictions = []

# Sequential prediction for specific horizons
for horizon in horizons:
    for _ in range(horizon - len(future_predictions)):  # Generate predictions for each horizon
        prediction = best_model.predict(current_input, verbose=0)  # Make the next step prediction
        future_predictions.append(prediction[0, 0])  # Append predicted value to the list
        # Update the input for the next prediction by shifting the sequence
        current_input = np.append(current_input[0, 1:], prediction).reshape(1, -1, 1)

# Inverse scaling to get the actual prediction values
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Calculate cumulative totals for different time horizons
next_hour = round(future_predictions[0][0])  # Single prediction for the next hour
next_day = round(sum(future_predictions[:24, 0]))  # Total for the first 24 predictions (24 hours)
next_week = round(sum(future_predictions[:24 * 7, 0]))  # Total for the first 7 days (168 hours)
next_month = round(sum(future_predictions[:24 * 30, 0]))  # Total for the first 30 days (720 hours)

# Print the future predictions for different time horizons
print(f"Next Hour Prediction (Calls): {next_hour}")
print(f"Total Calls Next Day (Day 1, 24 hours): {next_day}")
print(f"Total Calls Next Week (Days 1-7): {next_week}")
print(f"Total Calls Next Month (Days 1-30): {next_month}")


Best Model: LSTM


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Next Hour Prediction (Calls): 1
Total Calls Next Day (Day 1, 24 hours): 14
Total Calls Next Week (Days 1-7): 103
Total Calls Next Month (Days 1-30): 443


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf

# Load the dataset (replace with your file path)
data = pd.read_csv("top_A4_usage.csv")  # Replace with actual file
data['Time of Call'] = pd.to_datetime(data['Time of Call'],dayfirst=True)
data.set_index('Time of Call', inplace=True)

# Aggregate API calls by time (e.g., hourly)
time_series = data.resample('H').count()['API Code']

# Normalize the data
scaler = MinMaxScaler()
time_series_scaled = scaler.fit_transform(time_series.values.reshape(-1, 1))

# Create supervised learning dataset
def create_supervised(data, lag=24):
    X, y = [], []
    for i in range(lag, len(data)):
        X.append(data[i-lag:i])
        y.append(data[i])
    return np.array(X), np.array(y)

lag = 24  # Use the last 24 hours to predict the next step
X, y = create_supervised(time_series_scaled, lag)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


<ipython-input-14-34265028b870>:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_series = data.resample('H').count()['API Code']


In [15]:
from tensorflow.keras.layers import LSTM, Conv1D, GlobalAveragePooling1D, Dropout, concatenate

def build_lstm_fcn(input_shape):
    # LSTM branch
    input_layer = tf.keras.Input(shape=input_shape)
    lstm_out = LSTM(64)(input_layer)

    # FCN branch
    conv1 = Conv1D(128, kernel_size=8, activation='relu', padding='same')(input_layer)
    conv2 = Conv1D(256, kernel_size=5, activation='relu', padding='same')(conv1)
    conv3 = Conv1D(128, kernel_size=3, activation='relu', padding='same')(conv2)
    gap = GlobalAveragePooling1D()(conv3)

    # Merge branches
    merged = concatenate([lstm_out, gap])
    output_layer = Dense(1)(merged)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse')
    return model


In [16]:
def build_gru_fcn(input_shape):
    # GRU branch
    input_layer = tf.keras.Input(shape=input_shape)
    gru_out = GRU(64)(input_layer)

    # FCN branch
    conv1 = Conv1D(128, kernel_size=8, activation='relu', padding='same')(input_layer)
    conv2 = Conv1D(256, kernel_size=5, activation='relu', padding='same')(conv1)
    conv3 = Conv1D(128, kernel_size=3, activation='relu', padding='same')(conv2)
    gap = GlobalAveragePooling1D()(conv3)

    # Merge branches
    merged = concatenate([gru_out, gap])
    output_layer = Dense(1)(merged)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse')
    return model


In [17]:
pip install PyWavelets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 34.2 MB/s eta 0:00:00


In [18]:

import pywt

def wavelet_decomposition(data, wavelet='db1', level=3):
    coeffs = pywt.wavedec(data, wavelet, level=level)
    reconstructed = pywt.waverec(coeffs, wavelet)
    return np.array(coeffs).T  # Transpose for model input

# Example model
def build_mwdn(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)
    x = Conv1D(64, kernel_size=3, activation='relu', padding='same')(input_layer)
    x = LSTM(64, return_sequences=True)(x)
    x = GlobalAveragePooling1D()(x)
    output_layer = Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse')
    return model


In [19]:
from tensorflow.keras.layers import LayerNormalization, Activation

def build_tcn(input_shape, nb_filters=64, kernel_size=3, nb_stacks=1, dilation_rates=[1, 2, 4, 8]):
    input_layer = tf.keras.Input(shape=input_shape)
    x = input_layer

    for _ in range(nb_stacks):
        for dilation_rate in dilation_rates:
            x = Conv1D(nb_filters, kernel_size, padding="causal", dilation_rate=dilation_rate)(x)
            x = LayerNormalization()(x)
            x = Activation('relu')(x)

    x = GlobalAveragePooling1D()(x)
    output_layer = Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse')
    return model


In [20]:
def build_mlstm_fcn(input_shape):
    # LSTM branch
    input_layer = tf.keras.Input(shape=input_shape)
    lstm_out = LSTM(64, return_sequences=True)(input_layer)
    lstm_out = LSTM(32)(lstm_out)

    # FCN branch
    conv1 = Conv1D(128, kernel_size=8, activation='relu', padding='same')(input_layer)
    conv2 = Conv1D(256, kernel_size=5, activation='relu', padding='same')(conv1)
    conv3 = Conv1D(128, kernel_size=3, activation='relu', padding='same')(conv2)
    gap = GlobalAveragePooling1D()(conv3)

    # Merge branches
    merged = concatenate([lstm_out, gap])
    output_layer = Dense(1)(merged)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse')
    return model


In [21]:
from math import sqrt

def train_and_evaluate(model_builder, X_train, y_train, X_test, y_test, model_name):
    model = model_builder(X_train.shape[1:])
    history = model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0, validation_data=(X_test, y_test))
    y_pred = model.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    print(f"{model_name} RMSE: {rmse}")
    return model, rmse

# Train and evaluate all models
models = {
    "LSTM_FCN": build_lstm_fcn,
    "GRU-FCN": build_gru_fcn,
    "mWDN": build_mwdn,
    "TCN ": build_tcn,
    "MLSTM-FCN": build_mlstm_fcn,
}

results = {}
for name, builder in models.items():
    model, rmse = train_and_evaluate(builder, X_train, y_train, X_test, y_test, name)
    results[name] = rmse

# Print results
print("Model Performance:")
for model_name, rmse in sorted(results.items(), key=lambda x: x[1]):
    print(f"{model_name}: RMSE = {rmse}")


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
LSTM_FCN RMSE: 0.15152736916406825
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
GRU-FCN RMSE: 0.1519717502606059
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step
mWDN RMSE: 0.1507166762902506
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
TCN  RMSE: 0.15325445536788024
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step
MLSTM-FCN RMSE: 0.15565109540219674
Model Performance:
mWDN: RMSE = 0.1507166762902506
LSTM_FCN: RMSE = 0.15152736916406825
GRU-FCN: RMSE = 0.1519717502606059
TCN : RMSE = 0.15325445536788024
MLSTM-FCN: RMSE = 0.15565109540219674


In [22]:
# Select the best model
best_model_name = min(results, key=results.get)
print(f"Best Model: {best_model_name}")

# Retrain the best model on the entire dataset
best_model = models[best_model_name](X_train.shape[1:])
best_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)

# Start with the last observed input
current_input = X_test[-1].reshape(1, -1, 1)

# Define time horizons
horizons = [1, 24, 24 * 7, 24 * 30]  # Next hour, day, week, year

# Sequential prediction for specific horizons
future_predictions = []
for horizon in horizons:
    for _ in range(horizon - len(future_predictions)):
        prediction = best_model.predict(current_input, verbose=0)
        future_predictions.append(prediction[0, 0])
        current_input = np.append(current_input[0, 1:], prediction).reshape(1, -1, 1)



future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Calculate cumulative totals
next_hour = round(future_predictions[0][0])  # Single prediction for the next hour
next_day = round(sum(future_predictions[:24, 0]))  # Total for the first 24 predictions (Day 1)
next_week = round(sum(future_predictions[:24 * 7, 0]))  # Total for Days 1-7 (Week)
next_month = round(sum(future_predictions[:24 * 30, 0]))  # Total for Days 1-30 (Month)

# Print results
print(f"Next Hour Prediction (Calls): {next_hour}")
print(f"Total Calls Next Day (Day 1, 24 hours): {next_day}")
print(f"Total Calls Next Week (Days 1-7): {next_week}")
print(f"Total Calls Next Month (Days 1-30): {next_month}")


Best Model: mWDN
Next Hour Prediction (Calls): 1
Total Calls Next Day (Day 1, 24 hours): 16
Total Calls Next Week (Days 1-7): 104
Total Calls Next Month (Days 1-30): 444


In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf

# Load the dataset (replace with your file path)
data = pd.read_csv("top_A7_usage.csv")  # Replace with actual file
data['Time of Call'] = pd.to_datetime(data['Time of Call'],dayfirst=True)
data.set_index('Time of Call', inplace=True)

# Aggregate API calls by time (e.g., hourly)
time_series = data.resample('H').count()['API Code']

# Normalize the data
scaler = MinMaxScaler()
time_series_scaled = scaler.fit_transform(time_series.values.reshape(-1, 1))

# Create supervised learning dataset
def create_supervised(data, lag=24):
    X, y = [], []
    for i in range(lag, len(data)):
        X.append(data[i-lag:i])
        y.append(data[i])
    return np.array(X), np.array(y)

lag = 24  # Use the last 24 hours to predict the next step
X, y = create_supervised(time_series_scaled, lag)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


<ipython-input-23-6e7d9c243c4b>:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_series = data.resample('H').count()['API Code']


In [25]:
import tensorflow as tf
from tensorflow.keras import layers

def build_inceptiontime(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)

    # First branch: 1x1 convolution
    branch1 = layers.Conv1D(32, 1, padding='same', activation='relu')(input_layer)

    # Second branch: 3x3 convolution
    branch2 = layers.Conv1D(32, 3, padding='same', activation='relu')(input_layer)

    # Third branch: 5x5 convolution
    branch3 = layers.Conv1D(32, 5, padding='same', activation='relu')(input_layer)

    # Concatenate all branches
    concatenated = layers.concatenate([branch1, branch2, branch3])

    # Global average pooling and dense layer
    x = layers.GlobalAveragePooling1D()(concatenated)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)  # Change to 'Dense(num_classes, activation="softmax")' for classification

    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [26]:
def build_xceptiontime(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)

    # Xception block
    x = layers.Conv1D(32, 3, padding='same', activation='relu')(input_layer)
    x = layers.SeparableConv1D(64, 3, padding='same', activation='relu')(x)
    x = layers.SeparableConv1D(128, 3, padding='same', activation='relu')(x)

    # Global average pooling and dense layer
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)  # For regression, change for classification

    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [27]:
def build_rescnn(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)

    # First convolutional block with residual connections
    x = layers.Conv1D(32, 3, padding='same', activation='relu')(input_layer)
    x = layers.Conv1D(32, 3, padding='same', activation='relu')(x)
    residual = layers.Conv1D(32, 1, padding='same')(input_layer)  # Residual connection
    x = layers.add([x, residual])

    # Second convolutional block
    x = layers.Conv1D(64, 3, padding='same', activation='relu')(x)
    x = layers.Conv1D(64, 3, padding='same', activation='relu')(x)
    residual = layers.Conv1D(64, 1, padding='same')(x)  # Residual connection
    x = layers.add([x, residual])

    # Global average pooling and dense layer
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)  # For regression, change for classification

    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [28]:

def build_xcm(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)
    x = layers.Conv1D(64, 3, activation='relu', padding='same')(input_layer)
    x = layers.Conv1D(128, 3, activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [29]:
from tensorflow.keras import layers, models

def build_tst(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)
    x = layers.MultiHeadAttention(num_heads=8, key_dim=64)(input_layer, input_layer)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [30]:
from math import sqrt

def train_and_evaluate(model_builder, X_train, y_train, X_test, y_test, model_name):
    model = model_builder(X_train.shape[1:])
    history = model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0, validation_data=(X_test, y_test))
    y_pred = model.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    print(f"{model_name} RMSE: {rmse}")
    return model, rmse

# Train and evaluate all models
models = {
    "InceptionTime": build_inceptiontime,
    "XceptionTime": build_xceptiontime,
    "ResCNN": build_rescnn,
    "TST": build_tst,
    "XCM": build_xcm,

}

results = {}
for name, builder in models.items():
    model, rmse = train_and_evaluate(builder, X_train, y_train, X_test, y_test, name)
    results[name] = rmse

# Print results
print("Model Performance:")
for model_name, rmse in sorted(results.items(), key=lambda x: x[1]):
    print(f"{model_name}: RMSE = {rmse}")


24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
InceptionTime RMSE: 0.19238481512475636
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
XceptionTime RMSE: 0.19082061590736465
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
ResCNN RMSE: 0.19146830757871952
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step
TST RMSE: 0.1908141198645721
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
XCM RMSE: 0.19245765850154187
Model Performance:
TST: RMSE = 0.1908141198645721
XceptionTime: RMSE = 0.19082061590736465
ResCNN: RMSE = 0.19146830757871952
InceptionTime: RMSE = 0.19238481512475636
XCM: RMSE = 0.19245765850154187


In [31]:
# Select the best model
best_model_name = min(results, key=results.get)
print(f"Best Model: {best_model_name}")

# Retrain the best model on the entire dataset
best_model = models[best_model_name](X_train.shape[1:])
best_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)

# Start with the last observed input
current_input = X_test[-1].reshape(1, -1, 1)

# Define time horizonsa
horizons = [1, 24, 24 * 7, 24 * 30]  # Next hour, day, week, year

# Sequential prediction for specific horizons
future_predictions = []
for horizon in horizons:
    for _ in range(horizon - len(future_predictions)):
        prediction = best_model.predict(current_input, verbose=0)
        future_predictions.append(prediction[0, 0])
        current_input = np.append(current_input[0, 1:], prediction).reshape(1, -1, 1)



future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Calculate cumulative totals
next_hour = round(future_predictions[0][0])  # Single prediction for the next hour
next_day = round(sum(future_predictions[:24, 0]))  # Total for the first 24 predictions (Day 1)
next_week = round(sum(future_predictions[:24 * 7, 0]))  # Total for Days 1-7 (Week)
next_month = round(sum(future_predictions[:24 * 30, 0]))  # Total for Days 1-30 (Month)

# Print results
print(f"Next Hour Prediction (Calls): {next_hour}")
print(f"Total Calls Next Day (Day 1, 24 hours): {next_day}")
print(f"Total Calls Next Week (Days 1-7): {next_week}")
print(f"Total Calls Next Month (Days 1-30): {next_month}")


Best Model: TST
Next Hour Prediction (Calls): 1
Total Calls Next Day (Day 1, 24 hours): 16
Total Calls Next Week (Days 1-7): 111
Total Calls Next Month (Days 1-30): 477


In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf

# Load the dataset (replace with your file path)
data = pd.read_csv("top_A9_usage.csv")  # Replace with actual file
data['Time of Call'] = pd.to_datetime(data['Time of Call'],dayfirst=True)
data.set_index('Time of Call', inplace=True)

# Aggregate API calls by time (e.g., hourly)
time_series = data.resample('H').count()['API Code']

# Normalize the data
scaler = MinMaxScaler()
time_series_scaled = scaler.fit_transform(time_series.values.reshape(-1, 1))

# Create supervised learning dataset
def create_supervised(data, lag=24):
    X, y = [], []
    for i in range(lag, len(data)):
        X.append(data[i-lag:i])
        y.append(data[i])
    return np.array(X), np.array(y)

lag = 24  # Use the last 24 hours to predict the next step
X, y = create_supervised(time_series_scaled, lag)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


<ipython-input-32-037b1aba7044>:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_series = data.resample('H').count()['API Code']


In [33]:
from tensorflow.keras import layers, models

def build_tabtransformer(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)
    x = layers.MultiHeadAttention(num_heads=8, key_dim=64)(input_layer, input_layer)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model



In [34]:
from tensorflow.keras import layers, models

def build_tsit(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)
    x = layers.Conv1D(64, 3, activation='relu', padding='same')(input_layer)
    x = layers.LayerNormalization()(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model



In [35]:
from tensorflow.keras import layers, models

def build_gmlp(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)
    x = layers.Dense(128, activation='relu')(input_layer)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model



In [36]:
from tensorflow.keras import layers, models

def build_tsperceiver(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)
    x = layers.MultiHeadAttention(num_heads=8, key_dim=64)(input_layer, input_layer)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [37]:
def build_gated_tabtransformer(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)
    x = layers.MultiHeadAttention(num_heads=8, key_dim=64)(input_layer, input_layer)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [38]:
from tensorflow.keras import layers, models

def build_tssequencerplus(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)
    x = layers.LSTM(128, return_sequences=True)(input_layer)
    x = layers.LSTM(64)(x)
    output = layers.Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [39]:
def build_patchtst(input_shape):
    input_layer = tf.keras.Input(shape=input_shape)
    x = layers.Conv1D(64, 3, activation='relu', padding='same')(input_layer)
    x = layers.MultiHeadAttention(num_heads=8, key_dim=64)(x, x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [40]:
from math import sqrt

def train_and_evaluate(model_builder, X_train, y_train, X_test, y_test, model_name):
    model = model_builder(X_train.shape[1:])
    history = model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0, validation_data=(X_test, y_test))
    y_pred = model.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    print(f"{model_name} RMSE: {rmse}")
    return model, rmse

# Train and evaluate all models
#


models = {


     "PatchTST": build_patchtst,
    "TabTransformer": build_tabtransformer,
    "TSPerceiver": build_tsperceiver,
    "GatedTabTransformer": build_gated_tabtransformer,
    "TSSequencerPlus": build_tssequencerplus
}

results = {}
for name, builder in models.items():
    model, rmse = train_and_evaluate(builder, X_train, y_train, X_test, y_test, name)
    results[name] = rmse

# Print results
print("Model Performance:")
for model_name, rmse in sorted(results.items(), key=lambda x: x[1]):
    print(f"{model_name}: RMSE = {rmse}")


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
PatchTST RMSE: 0.15860510679150563
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
TabTransformer RMSE: 0.15808687220778986
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step
TSPerceiver RMSE: 0.15829927685658812
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
GatedTabTransformer RMSE: 0.15822544695369975
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step
TSSequencerPlus RMSE: 0.15857893262855827
Model Performance:
TabTransformer: RMSE = 0.15808687220778986
GatedTabTransformer: RMSE = 0.15822544695369975
TSPerceiver: RMSE = 0.15829927685658812
TSSequencerPlus: RMSE = 0.15857893262855827
PatchTST: RMSE = 0.15860510679150563


In [41]:
# Select the best model
best_model_name = min(results, key=results.get)
print(f"Best Model: {best_model_name}")

# Retrain the best model on the entire dataset
best_model = models[best_model_name](X_train.shape[1:])
best_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)

# Start with the last observed input
current_input = X_test[-1].reshape(1, -1, 1)

# Define time horizons
horizons = [1, 24, 24 * 7, 24 * 30]  # Next hour, day, week, year

# Sequential prediction for specific horizons
future_predictions = []
for horizon in horizons:
    for _ in range(horizon - len(future_predictions)):
        prediction = best_model.predict(current_input, verbose=0)
        future_predictions.append(prediction[0, 0])
        current_input = np.append(current_input[0, 1:], prediction).reshape(1, -1, 1)



future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Calculate cumulative totals
next_hour = round(future_predictions[0][0])  # Single prediction for the next hour
next_day = round(sum(future_predictions[:24, 0]))  # Total for the first 24 predictions (Day 1)
next_week = round(sum(future_predictions[:24 * 7, 0]))  # Total for Days 1-7 (Week)
next_month = round(sum(future_predictions[:24 * 30, 0]))  # Total for Days 1-30 (Month)

# Print results
print(f"Next Hour Prediction (Calls): {next_hour}")
print(f"Total Calls Next Day (Day 1, 24 hours): {next_day}")
print(f"Total Calls Next Week (Days 1-7): {next_week}")
print(f"Total Calls Next Month (Days 1-30): {next_month}")


Best Model: TabTransformer
Next Hour Prediction (Calls): 1
Total Calls Next Day (Day 1, 24 hours): 17
Total Calls Next Week (Days 1-7): 118
Total Calls Next Month (Days 1-30): 505
